# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [22]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [23]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [24]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'class_db', 'config', 'epa', 'establishments', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [25]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [26]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [28]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65efa587dfae3f2f16f85a56'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [29]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [33]:
# Create a dictionary for the new restaurant data
greenwich = {
        'AddressLine3': 'Greenwich',
        'BusinessName': 'Penang Flavours',
        'BusinessType': '',
        'BusinessTypeID': ''
    }


In [34]:
# Insert the new restaurant into the collection
db.establishments.insert_one(greenwich)

InsertOneResult(ObjectId('65efa83d0d919f27d6b4a3d4'), acknowledged=True)

In [38]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
pprint(results[0])

{'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': '',
 'BusinessTypeID': '',
 '_id': ObjectId('65efa7c50d919f27d6b4a3d3')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [46]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':"Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}

# Capture the results to a variable
results = establishments.find(query, fields)

# Pretty print the results
for result in results:
    pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a57')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a58')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a5e')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a61')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a62')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a63')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a64')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f85a6a')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa587dfae3f2f16f8

3. Update the new restaurant with the `BusinessTypeID` you found.

In [48]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({'BusinessName': 'Penang Flavours'},
                             {'$set':
                            {'BusinessType': 'Restaurant/Cafe/Canteen',
                            'BusinessTypeID': 1}})



UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [49]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
pprint(results[0])

{'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65efa7c50d919f27d6b4a3d3')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [53]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_documents = establishments.count_documents({'LocalAuthorityName': 'Dover'})
dover_documents

0

In [51]:
# Delete all documents where LocalAuthorityName is "Dover"
db.establishments.delete_many(
    {'LocalAuthorityName': 'Dover'}
)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [54]:
# Check if any remaining documents include Dover
dover_documents = establishments.count_documents({'LocalAuthorityName': 'Dover'})
dover_documents

0

In [55]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65efa587dfae3f2f16f85d3a'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [56]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many({}, [
    {'$set':
        {
            "geocode.latitude": {'$toDecimal': "$geocode.latitude"},
            "geocode.longitude": {'$toDecimal': "$geocode.longitude"}
        }
    }
])

UpdateResult({'n': 38787, 'nModified': 38787, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [61]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [63]:
# Change the data type from String to Integer for RatingValue
db.establishments.update_many(
    {},
    [
        {
            '$set': {
                "RatingValue": {'$toInt': "$RatingValue"}
            }
        }
    ]
)


UpdateResult({'n': 38787, 'nModified': 34696, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)